In [ ]:
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

from simpletransformers.classification import ClassificationModel
import pandas as pd
from sklearn import preprocessing

In [ ]:
input_file = "./data/corpus.csv"
corpus = pd.read_csv("./data/corpus.csv", dtype="string")

In [ ]:
corpus.head()

In [ ]:
le = preprocessing.LabelEncoder()
corpus["y2"]=pd.Series(le.fit_transform(corpus["y2"]))
corpus["y6"]=pd.Series(le.fit_transform(corpus["y6"]))
corpus["y8"]=pd.Series(le.fit_transform(corpus["y8"]))

In [ ]:
corpus.head()

### SR data

In [ ]:
SR_data = corpus[corpus.NaturalLanguageID == "SR"]
# Remove IDE to be consistent with other models.
SR_data = SR_data[SR_data.y8 != "ide"]

### EN data

In [ ]:
EN_data = corpus[corpus.NaturalLanguageID == "EN"]

### Evaluation

In [ ]:
def make_score_name(score_name, model_name, num_classes):
    return f"{score_name}-{model_name}-{num_classes}"

In [ ]:
def f1_macro_score(y, y_pred):
    return f1_score(y, y_pred, average="macro")

In [ ]:
def write_results(result_file, score_name, score_value):
    pd.DataFrame(
        {"score_name": [score_name],
        "score_value": [score_value]}
    ).to_csv(result_file, mode="a", decimal=",", header=False, index=False)

In [ ]:
def evaluate(data_base_lang, data_test_lang, result_file, model_type, model_name, num_epochs, y_column_names=["y8","y6", "y2"], seeds=[11, 17, 23, 47, 62]):
    parameter_dict = {}
    parameter_dict["fp16"] = False
    parameter_dict["overwrite_output_dir"] = True
    parameter_dict["reprocess_input_data"] = True
    parameter_dict["no_cache"] = True
    parameter_dict["save_eval_checkpoints"] = False
    parameter_dict["save_model_every_epoch"] = False
    parameter_dict["use_cached_eval_features"] = False
    parameter_dict["output_dir"] = f"./Transformers/{model_name}/outputs/"
    parameter_dict["cache_dir"] = f"./Transformers/{model_name}/cache/"
    parameter_dict["tensorboard_dir"] = f"./Transformers/{model_name}/runs/"
    parameter_dict["silent"] = True
    parameter_dict["num_train_epochs"] = num_epochs
    parameter_dict["max_seq_length"] = 512

    X_base_lang = data_base_lang["Comment"].astype(str)
    X_test_lang = data_test_lang["Comment"].astype(str)

    for y_column_name in y_column_names:
        seed_scores = []
        
        for manual_seed in seeds:
            parameter_dict["manual_seed"] = manual_seed

            y_base_lang = data_base_lang[y_column_name]
            y_test_lang = data_test_lang[y_column_name]

            num_classes = y_base_lang.nunique()

            score_name = make_score_name(f"epochs{num_epochs}", model_name, num_classes)
            print(f"-------------------RUNNING {score_name}-seed{manual_seed} with {num_classes} classes.-------------------")

            cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
            i = 0
            f1_scores = []
            for train_index, test_index in cv.split(X_test_lang, y_test_lang):
                i += 1
                print(f"-------------------RUNNING at index {i}-------------------")

                # Prepare data.
                X_train, X_test = X_test_lang.iloc[train_index], X_test_lang.iloc[test_index]
                y_train, y_test = y_test_lang.iloc[train_index], y_test_lang.iloc[test_index]
            
                # print("X_train shape", X_train.shape, "y_train_Shape", y_train.shape, X_train.head())

                X_train = pd.concat([X_base_lang, X_train], ignore_index=True)
                y_train = pd.concat([y_base_lang, y_train], ignore_index=True)

                print("X_train shape", X_train.shape, "y_train shape", y_train.shape)

                train_df = pd.DataFrame(list(zip(X_train, y_train)), columns=['text', 'labels'])
                eval_df = pd.DataFrame(list(zip(X_test, y_test)), columns=['text', 'labels'])

                # Create model.
                model = ClassificationModel(model_type, model_name, num_labels=y_train.nunique(), use_cuda=True, args=parameter_dict)  # You can set class weights by using the optional weight argument
                # Train model.
                global_step, training_details = model.train_model(train_df, show_running_loss=False, verbose=False)
                print(global_step, training_details)
                # Evaluate model.
                print(f"-------------------EVALUATE model-------------------")
                result, y_pred, wrong_predictions = model.eval_model(eval_df, f1=f1_macro_score, verbose=False)

                # Get results.
                print("RESULT ", result)
                macro_f1 = result["f1"]
                f1_scores.append(macro_f1)

                # Write result.
                write_results(result_file, f"{i}-seed{manual_seed}-{score_name}", macro_f1)

            # Write mean result for a single seed.
            means_f1_score = sum(f1_scores) / len(f1_scores)
            seed_scores.append(means_f1_score)
            write_results(result_file, f"0-seed{manual_seed}-{score_name}", means_f1_score)
        
        # Write mean result for all seeds.
        seeds_means_f1_score = sum(seed_scores) / len(seed_scores)
        write_results(result_file, f"0-mean-{score_name}", seeds_means_f1_score)

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

## Test SR

In [ ]:
result_file = "./results/transformers_multi_SR.csv"

### Multilingual bert

In [ ]:
evaluate(data_base_lang=EN_data, data_test_lang=SR_data, result_file=result_file, model_type="bert", model_name="bert-base-multilingual-cased", num_epochs=1)
evaluate(data_base_lang=EN_data, data_test_lang=SR_data, result_file=result_file, model_type="bert", model_name="bert-base-multilingual-cased", num_epochs=3)
evaluate(data_base_lang=EN_data, data_test_lang=SR_data, result_file=result_file, model_type="bert", model_name="bert-base-multilingual-cased", num_epochs=5)

### Multilingual Roberta

In [ ]:
evaluate(data_base_lang=EN_data, data_test_lang=SR_data, result_file=result_file, model_type="xlmroberta", model_name="xlm-roberta-base", num_epochs=1)
evaluate(data_base_lang=EN_data, data_test_lang=SR_data, result_file=result_file, model_type="xlmroberta", model_name="xlm-roberta-base", num_epochs=3)
evaluate(data_base_lang=EN_data, data_test_lang=SR_data, result_file=result_file, model_type="xlmroberta", model_name="xlm-roberta-base", num_epochs=5)

## Test EN

In [ ]:
result_file = "./results/transformers_multi_EN.csv"

### Multilingual bert

In [ ]:
evaluate(data_base_lang=SR_data, data_test_lang=EN_data, result_file=result_file, model_type="bert", model_name="bert-base-multilingual-cased", num_epochs=1)
evaluate(data_base_lang=SR_data, data_test_lang=EN_data, result_file=result_file, model_type="bert", model_name="bert-base-multilingual-cased", num_epochs=3)
evaluate(data_base_lang=SR_data, data_test_lang=EN_data, result_file=result_file, model_type="bert", model_name="bert-base-multilingual-cased", num_epochs=5)

### Multilingual roberta

In [ ]:
evaluate(data_base_lang=SR_data, data_test_lang=EN_data, result_file=result_file, model_type="xlmroberta", model_name="xlm-roberta-base", num_epochs=1)
evaluate(data_base_lang=SR_data, data_test_lang=EN_data, result_file=result_file, model_type="xlmroberta", model_name="xlm-roberta-base", num_epochs=3)
evaluate(data_base_lang=SR_data, data_test_lang=EN_data, result_file=result_file, model_type="xlmroberta", model_name="xlm-roberta-base", num_epochs=5)

## Corrected Paired students t test

In [1]:
import numpy as np
from scipy.stats import t
def get_p_value(scores1, scores2, train_size, test_size):
    diff = [y-x for x, y in zip(scores1, scores2)]
    diff_mean = np.mean(diff)
    diff_var = np.var(diff)
    diff_var_mod = diff_var * (1/(train_size+test_size) + test_size/train_size)
    t_static = diff_mean / np.sqrt(diff_var_mod)
    p_value = (1-t.cdf(t_static, (test_size+train_size)-1))*200
    return p_value

English mono Electra vs multi BERT

In [2]:
mono_y8 = [0.776178012184384,0.800280994002721,0.808858694585637,0.809708285319853,0.791571485407733,0.810395838752313,0.810816949741258,0.811460626092077,0.775305806450374,0.798095467231081,0.779587270965963,0.800037543216307,0.764678405783935,0.776432684927734,0.772698779926291,0.794537487090517,0.823985637807982,0.804876663581035,0.785655101593946,0.81558610621327,0.802404335402579,0.796819850445721,0.800925865315933,0.786234671257989,0.81337521133012,0.806795231882436,0.850768780118292,0.779783045315889,0.807011243076606,0.815580472511553,0.763893307388767,0.797836406308927,0.806283694728816,0.787494925828903,0.815018075860964,0.811604186869739,0.847900985941503,0.782995340092141,0.817687361092707,0.792332517616895,0.795881856296678,0.819184619854207,0.797501068495481,0.811001856742746,0.795706427527554,0.796283501458153,0.836038740496419,0.788818183437777,0.783954005339819,0.783966836395345]
mono_y6 = [0.802515126247317,0.829186585883986,0.314933219664677,0.838841486754018,0.81565498501464,0.832025135159892,0.826954762224814,0.791636402345108,0.866875735166225,0.871849252532945,0.883310169818622,0.815668032217328,0.862873422004816,0.828169183228316,0.854136857488396,0.85939219187301,0.840690056666634,0.829827125692758,0.863059935156709,0.898210506967852,0.855249500995953,0.816537323532692,0.840088309727061,0.848031273274441,0.866185499051196,0.839309889859681,0.859815750860556,0.82182193473211,0.844044390018638,0.87478951939928,0.840148634560197,0.786152696475277,0.862233294205125,0.829530929294069,0.82305502901047,0.868769355665716,0.85952131298492,0.824712132150243,0.856436229173199,0.871084662489591,0.875639668248779,0.825389294098971,0.867276735973447,0.851744987132518,0.870225079124906,0.83043307514964,0.833694016784619,0.823296336232545,0.861866667692413,0.864711877056409]
mono_y2 = [0.888736117382635,0.919826534760807,0.909430887514624,0.904042731462086,0.914233960992459,0.930697528278173,0.891012685331703,0.909373690825303,0.915823498458994,0.911797602356684,0.884665200139799,0.900769629885852,0.922537461177074,0.895181772579665,0.916213991769547,0.922058241148982,0.890415202928002,0.913754394882873,0.901857113862391,0.909252986272062,0.897501172372211,0.900176159718144,0.919194693256717,0.899997345015266,0.916682947435119,0.908873583554488,0.887910151948909,0.917143202213855,0.902694463819783,0.906962048453893,0.875142593721188,0.90515053387012,0.908013872641728,0.905072734054457,0.903513206116517,0.915620208048406,0.916213991769547,0.911797602356684,0.914554305858653,0.918686632018403,0.89108183707593,0.902656363328988,0.919194693256717,0.900557066293528,0.905932360007392,0.91326574081963,0.89700306722331,0.902423958422084,0.926939796587926,0.924954715868402]

multi_y8 = [0.839133306915261,0.866414143783047,0.876934211997378,0.837398416769342,0.845223730915788,0.849881965893173,0.889647733358117,0.839968049768882,0.819604874298007,0.853841384148289,0.850646156670973,0.902003182835912,0.840552251665806,0.829673805389582,0.864918229072593,0.843946025806714,0.882057636774717,0.852688459931929,0.826765534916698,0.867722101688029,0.838457294288611,0.882793505845338,0.867368963636043,0.82893879422134,0.865221825662467,0.849090982442465,0.875113829883714,0.847462709544352,0.838095775170991,0.853124375945299,0.853259206504312,0.87736571528079,0.837893242191415,0.855871803785826,0.84368101913522,0.862256335254298,0.896006551108234,0.8521966698397,0.824448736830687,0.849213168640684,0.824550278110604,0.890447661942527,0.8527007328049,0.859965933409677,0.852919849428555,0.859326502079476,0.881675962914221,0.844738442528426,0.846883399399979,0.846303413819265]
multi_y6 = [0.913172764634967,0.888191763191763,0.898392324428593,0.900477531235858,0.899847065279452,0.921278301156822,0.914169910780559,0.886868964308854,0.911570252968279,0.922323213753672,0.890446622238375,0.889376872946444,0.917946079502683,0.887427314559918,0.906262607418802,0.924818396472161,0.897586809680105,0.892279589814801,0.916492730810014,0.919762245733926,0.918685673646763,0.874018814651385,0.888790277510473,0.910988148611991,0.899964863869507,0.926867420256482,0.926588279400959,0.883765016117957,0.902750611961138,0.92292962176017,0.92443840226409,0.881245832515269,0.937358976092999,0.89261140845242,0.894288562194869,0.921433745760575,0.913489275461106,0.895735701938806,0.901746996409233,0.902841936002381,0.929421176381382,0.879936720594518,0.905926409620149,0.913128011456918,0.920076735914795,0.924736175807733,0.917759952569648,0.901101097802747,0.925467249513668,0.898218920534297]
multi_y2 = [0.918954248366012,0.904921193784988,0.940180834408514,0.935675470744344,0.921619540687641,0.927025089605734,0.940706146481975,0.899260416666666,0.926557282433936,0.917744020685197,0.928659872862257,0.897066009475451,0.9221754257462,0.923671808054841,0.907844849620857,0.943241736360016,0.918038406997823,0.899260416666666,0.928614096417459,0.913754394882873,0.914172501542651,0.909306949900577,0.942633344240837,0.919594338430431,0.912278502827956,0.940370196813495,0.923671808054841,0.904042731462086,0.925770833333333,0.91326574081963,0.910342506692919,0.901923076923076,0.940180834408514,0.934949305614722,0.914233960992459,0.935315796162155,0.922313797313797,0.916065465638784,0.926167682042355,0.901106402642343,0.911798815845693,0.889744033206502,0.929411764705882,0.929528414415949,0.927025089605734,0.948934709718298,0.930697528278173,0.917143202213855,0.931072916666666,0.923671808054841]

In [3]:
total_size = 5093
print("y2 - {:.3}%".format(get_p_value(mono_y2, multi_y2, train_size=total_size*0.9, test_size=total_size*0.1)))
print("y6 - {:.3}%".format(get_p_value(mono_y6, multi_y6, train_size=total_size*0.9, test_size=total_size*0.1)))
print("y8 - {:.3}%".format(get_p_value(mono_y8, multi_y8, train_size=total_size*0.9, test_size=total_size*0.1)))

y2 - 1.39%
y6 - 0.415%
y8 - 0.0%


Serbian mono Bertic vs mult BERT

In [4]:
mono_y8 = [0.873204957085511,0.882598598269952,0.884905582328728,0.866719723496705,0.879643125053263,0.858889854120003,0.850015756907588,0.871678268821126,0.885027574474983,0.841022719713968,0.870720949291897,0.839761759577018,0.883228692393112,0.872637924749932,0.889251561665319,0.851601537314777,0.854855559600591,0.884483533191371,0.876362250192274,0.8579800609388,0.893133708176838,0.839067315045031,0.865564208221186,0.873677287869205,0.87062938623552,0.853553651522965,0.861331190681091,0.8748159539183,0.867520438387621,0.85202952353758,0.857508642107389,0.837894773144635,0.883364539424983,0.858787707187418,0.872630820459127,0.854502338696317,0.846044096948356,0.870118148200655,0.867504914469791,0.839177732358961,0.856663514322965,0.858392764644679,0.882955716602333,0.854013392522754,0.886604408370026,0.854058130591963,0.878098788328807,0.884093393274345,0.877539407419806,0.848047354227129]
mono_y6 = [0.873458333333333,0.915686571784132,0.89315407529658,0.873976501599655,0.895058456743435,0.885906763664116,0.895655873538226,0.89868017072832,0.906462296581293,0.900332885505299,0.867604166666666,0.906978954707414,0.913065735892961,0.874795908963057,0.912461008090131,0.882981953468185,0.879876476678014,0.862995431864502,0.904002317002833,0.917792532802241,0.854993948294689,0.906276463262764,0.913255875885916,0.875301061403641,0.89259316697369,0.884664378132289,0.888523483671691,0.89272555543134,0.904457300311303,0.888658541784847,0.86354044013797,0.897807537095672,0.90649942041396,0.892631823365653,0.91491897579639,0.854748713572242,0.87585080709785,0.908581048581048,0.922682356403286,0.877832706257653,0.845880824534441,0.907335188715768,0.928065157000483,0.881491770565735,0.889323130603869,0.909444793549808,0.860734410261425,0.8974007119005,0.915389172225009,0.897176093217975]
mono_y2 = [0.953648152255649,0.950206375691578,0.939141125845262,0.940185141229527,0.942706045073631,0.93194425652821,0.93910411622276,0.950176095091349,0.928862053959965,0.950176095091349,0.950496562261268,0.943692243851314,0.932811944543192,0.944996180290298,0.950466123949579,0.937998655311519,0.942042106302776,0.94363514119229,0.943028655566881,0.948454497893845,0.953381830448716,0.942741835139331,0.932811944543192,0.958866320984161,0.94099706744868,0.929327331074056,0.93874274832824,0.94957898957498,0.955712084525643,0.944640105657054,0.94084885730747,0.95078125,0.9246407309982,0.935823429541595,0.943028655566881,0.934981144588131,0.942377245476316,0.944311589389309,0.934334203655352,0.955712084525643,0.956498284530565,0.953909465020576,0.930222899072407,0.953909465020576,0.956470944571848,0.93194425652821,0.93910411622276,0.945278503046127,0.942042106302776,0.944962359943977]

multi_y8 = [0.868187655695027,0.861949314495268,0.864529930115146,0.875507838487751,0.788120421286544,0.732081384272477,0.72853933551527,0.831068567648919,0.905505821905804,0.826114793156618,0.875564271238191,0.858908206011744,0.856758926450367,0.87449463652683,0.777430645326581,0.737095707985653,0.75162228016784,0.884909320996824,0.836063240459547,0.855501428779914,0.890018148204578,0.860536854472428,0.888463871716071,0.86609297962603,0.771339318123207,0.860547507066494,0.737394214264964,0.890179144334251,0.89653982852285,0.846803697829787,0.876061155704241,0.858316718209206,0.866213205945031,0.766548181743125,0.771703625626941,0.847230557314133,0.872064819632328,0.871150841832933,0.885414704929581,0.859461688683702,0.888041857584604,0.746704764559098,0.876054923855818,0.874972034939338,0.794550642746365,0.799859471519322,0.861462626345586,0.879100963099779,0.853140918938913,0.834460410374821]
multi_y6 = [0.837573629664857,0.904706506173227,0.928410595350724,0.901877220722857,0.754140016514758,0.743801275679833,0.723435407217094,0.841199709512956,0.923845203020759,0.879260007745779,0.827073245766788,0.898976832087791,0.913002189381499,0.871605189676185,0.749241939802634,0.747860899168893,0.901297468877079,0.894032742173351,0.90567084523065,0.901749066818769,0.855512371205378,0.901126770898433,0.915496156357178,0.750669434549164,0.754735474687289,0.74082992453775,0.722546605820879,0.909736303123399,0.742789349546882,0.881443531741978,0.837944610037935,0.898011170692948,0.908856723783841,0.868902164655965,0.738806492170233,0.727277909150203,0.750575772430947,0.907499828901456,0.914864385946023,0.900843953259254,0.839705598799786,0.898120632585792,0.927921012942775,0.86748281877671,0.757407541008484,0.752898072759771,0.738929114049223,0.904588439226993,0.905484855607353,0.876750325336615]
multi_y2 = [0.936561099344924,0.949608958407394,0.94434506062413,0.896683673469387,0.935415935415935,0.938779566686543,0.94819499369749,0.924183924183924,0.92890625,0.89666812874426,0.934747426795848,0.952833668217224,0.950206375691578,0.919539786958795,0.952551571369237,0.947593951632763,0.931835518023013,0.935805067146614,0.92595937777149,0.906916493176035,0.931835518023013,0.955739000614736,0.955996944232238,0.904236056778808,0.931044673201004,0.923718124793291,0.93296695696585,0.928495034377387,0.922543251075788,0.895959735245449,0.929309712362169,0.958382648426963,0.94592738820702,0.91601493067899,0.941735707797748,0.947593951632763,0.9453125,0.933608500922104,0.9125,0.917509740828917,0.925528247057085,0.952551571369237,0.938779566686543,0.915502403040125,0.944996180290298,0.938779566686543,0.947593951632763,0.931044673201004,0.931044673201004,0.907519764662621]

In [5]:
total_size = 5039
print("y2 - {:.3}%".format(get_p_value(multi_y2, mono_y2, train_size=total_size*0.9, test_size=total_size*0.1)))
print("y6 - {:.3}%".format(get_p_value(multi_y6, mono_y6, train_size=total_size*0.9, test_size=total_size*0.1)))
print("y8 - {:.3}%".format(get_p_value(multi_y8, mono_y8, train_size=total_size*0.9, test_size=total_size*0.1)))

y2 - 9.39%
y6 - 3.16%
y8 - 11.1%
